In [ ]:
spark

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from google.cloud import storage
from io import BytesIO
from datetime import datetime, date

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, BooleanType, IntegerType , DateType, FloatType, StructType, StructField
from pyspark.sql.functions import col, isnan, when, count, udf, to_date, year, month, date_format, size, split, dayofweek
from pyspark.sql import functions as F

# Taxi Data Frame

In [ ]:
bucket_name = 'my-bigdataproject-jg'
gs_path  = f'gs://{bucket_name}/'
landing_folder = 'landing/'
cleaned_folder = 'cleaned/'
destination_folder = 'code_and_models/'

storage_client = storage.Client() 
bucket = storage_client.get_bucket(bucket_name)

In [ ]:
data_years = [2021, 2022, 2023]
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

taxi_file_path = gs_path + landing_folder
taxi_file_list = [taxi_file_path + f'yellow_tripdata_{year}-{month}.parquet' for year in data_years for month in months]

taxi_df = None
for file in taxi_file_list:
    df = spark.read.parquet(file)
    df = df.withColumn('VendorID', df['VendorID'].cast(IntegerType()))
    df = df.withColumn('passenger_count', df['passenger_count'].cast(IntegerType()))
    if taxi_df is None:
        taxi_df = df
    else:
        taxi_df = taxi_df.union(df)

taxi_df.show(5)

In [ ]:
print(f'Number of records in the dataframe: {taxi_df.count():,}')

# Date Range

In [ ]:
taxi_df = taxi_df.filter((col('tpep_pickup_datetime') >= '2021-01-01') & (col('tpep_pickup_datetime') < '2024-01-01'))
taxi_df = taxi_df.filter((col('tpep_dropoff_datetime') >= '2021-01-01') & (col('tpep_dropoff_datetime') < '2024-01-02'))

taxi_df.select(F.min('tpep_pickup_datetime'), F.max('tpep_pickup_datetime')).show()
taxi_df.select(F.min('tpep_dropoff_datetime'), F.max('tpep_dropoff_datetime')).show()

# rename tpep_pickup_datetime and tpep_dropoff_datetime columns to pickup_datetime and dropoff_datetime
taxi_df = taxi_df.withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime')
taxi_df = taxi_df.withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')


# payment_type

In [ ]:
# payment_type = 6 are voided trips
taxi_df = taxi_df.filter(col('payment_type') != 6)

# RatecodeID

In [ ]:
# 1 is standard rate, 2 is JFK, 6 is group ride, 99 is unknown
allowed_rate_codes = [1, 2, 6, 99]
# fill the empty RatecodeID with 99
taxi_df = taxi_df.withColumn('RatecodeID', when(col('RatecodeID').isNull(), 99).otherwise(col('RatecodeID')))
# Change RatecodeID to an integer
taxi_df = taxi_df.withColumn('RatecodeID', taxi_df['RatecodeID'].cast(IntegerType()))
# filter the RatecodeID to only include the allowed values
taxi_df = taxi_df.filter(col('RatecodeID').isin(allowed_rate_codes))

# passenger_count

In [ ]:
taxi_df = taxi_df.fillna({'passenger_count': 0})
taxi_df = taxi_df.filter(col('passenger_count') <= 7)


In [ ]:
print(f'Number of records in the dataframe: {taxi_df.count():,}')

# fare_amount

In [ ]:

taxi_df = taxi_df.filter((col('fare_amount') >= 3) & (col('fare_amount') <= 250))


# total_amount

In [ ]:

taxi_df = taxi_df.filter((col('total_amount') > 3) & (col('total_amount') < 400))

# tip_amount

In [ ]:
# tip amount should be less than 150 and greater than 0
taxi_df = taxi_df.filter((col('tip_amount') >= 0) & (col('tip_amount') < 150))

# tip_percentage

In [ ]:
# Create a tip percentage column
taxi_df = taxi_df.withColumn('tip_percentage', col('tip_amount') / col('total_amount') * 100)

# trip_distance

In [ ]:
# trip_distance should be greater than 0 and less than 60
taxi_df = taxi_df.filter((col('trip_distance') > 0) & (col('trip_distance') < 60))

# Saving the cleaned taxi data

In [ ]:
# Summarize the fare_amount, tip_amount, total_amount, tip_percentage, trip_distance
taxi_df.select('fare_amount', 'tip_amount', 'total_amount', 'tip_percentage', 'trip_distance').summary().show()

In [ ]:
# Schema for the cleaned data
taxi_df = taxi_df.select('pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'PULocationID', 'DOLocationID', 'fare_amount', 'tip_amount', 'total_amount', 'tip_percentage')
taxi_df.printSchema()

In [ ]:
# save the cleaned data
print('Saving the cleaned data')
taxi_df.write.mode('overwrite').parquet(gs_path + cleaned_folder + 'taxi_data/')
print('Data saved successfully')